In [1]:
%pylab
from numba import (
    jit,
    vectorize,
    float32,
    float64,
    cfunc,
    njit,
    prange,
    get_num_threads,
    set_num_threads,
)
import numpy as np
from math import sqrt
from scipy.special import comb
from scipy.interpolate import interp2d, RectBivariateSpline
from numba import cuda
from numpy import float64, float32, int32, ndarray

cuda.detect()


# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit("void(float32[:], float32[:,:], float32[:], float32[:,:], float32)",fastmath=True)
def GridSurfaceDensity_core_cuda(f, x2d, h, grid, size):
    res = np.int32(grid.shape[0])
    dx = np.float32(size / (res - 1))

    # numba provides this function for working out which element you're
    # supposed to be accessing
    i = cuda.grid(1)
    if i<f.shape[0]: # and j<n3.shape[1]: # check we're in range
       # print(i)
        # do work on a single element
        xs = x2d[i]
        hs = h[i]
        hs_sqr = hs*hs
        hinv = 1 / hs
        mh2 = f[i] * hinv * hinv * 1.8189136353359467

        gxmin = max(int((xs[0] - hs) / dx + 1), 0)
        gxmax = min(int((xs[0] + hs) / dx), res - 1)
        gymin = max(int((xs[1] - hs) / dx +   1), 0)
        gymax = min(int((xs[1] + hs) / dx), res - 1)

        for gx in range(gxmin, gxmax + 1):
            delta_x_Sqr = xs[0] - gx * dx
            delta_x_Sqr *= delta_x_Sqr
            for gy in range(gymin, gymax + 1):
                delta_y_Sqr = xs[1] - gy * dx
                delta_y_Sqr *= delta_y_Sqr
                r = delta_x_Sqr + delta_y_Sqr
                if r > hs_sqr:
                    continue
                r = sqrt(r)
                q = r * hinv
                if q <= 0.5:
                    kernel = 1 - 6 * q * q * (1 - q)
                else: # q <= 1.0:
                    a = 1 - q
                    kernel = 2 * a * a * a
                cuda.atomic.add(grid, (gx,gy), kernel * mh2)
       #cuda.syncthreads()
                


Using matplotlib backend: <object object at 0x7fee42d90350>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Found 1 CUDA devices
id 0         b'NVIDIA T1000'                              [SUPPORTED]
                      Compute Capability: 7.5
                           PCI Device ID: 0
                              PCI Bus ID: 101
                                    UUID: GPU-b303fbe2-bd8d-69ed-9a8c-01198eed12ed
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
Summary:
	1/1 devices are supported


In [9]:
TPB = 16
import math
N = 2048
grid = cuda.device_array((N, N), dtype=np.float32)
#grid = np.zeros((N,N))
#grid[:] = 0

Np = 10**6
x = np.float32(np.random.rand(Np,3))
x = x[x[:,0].argsort()]
x = x[x[:,1].argsort()]
m = np.float32(np.repeat(1./Np,Np))
h = np.float32(np.repeat(Np**(-1./3),Np))*2
center = np.float32([0.5,0.5,0.5])
size = np.float32(1.)

threadsperblock = 16
blockspergrid = int(ceil(Np // threadsperblock))
md = cuda.to_device(m)
xd = cuda.to_device(x - center + size / 2)
hd = cuda.to_device(h)
%time GridSurfaceDensity_core_cuda[blockspergrid,threadsperblock](md,xd,hd,grid,size), grid.copy_to_host()

CPU times: user 10.6 s, sys: 76.1 ms, total: 10.7 s
Wall time: 7.37 s


(None,
 array([[0.30436262, 0.3243586 , 0.3440726 , ..., 0.31730238, 0.30075228,
         0.28387037],
        [0.32386306, 0.34512836, 0.36608908, ..., 0.3391451 , 0.3214441 ,
         0.30338868],
        [0.34302232, 0.36553156, 0.3877163 , ..., 0.36100018, 0.34214982,
         0.32292193],
        ...,
        [0.28866827, 0.3068082 , 0.3246588 , ..., 0.32269472, 0.30338594,
         0.28401226],
        [0.27150697, 0.28856844, 0.30535436, ..., 0.30480537, 0.28652766,
         0.26819173],
        [0.25427198, 0.27025187, 0.28597048, ..., 0.28670752, 0.269474  ,
         0.25218907]], dtype=float32))

In [4]:
from meshoid import Meshoid
from meshoid.backend import GridSurfaceDensity
#GridSurfaceDensity()

%timeit Meshoid(x,m,h).SurfaceDensity(m,center=np.array([0.5,0.5,0.5]),size=1.,res=1024)

5.81 s ± 409 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%pylab
Np = 10**6
x = np.float32(np.random.rand(Np,3))
x = x[x[:,0].argsort()]
x = x[x[:,1].argsort()]
m = np.float32(np.repeat(1./Np,Np))
h = np.float32(np.repeat(Np**(-1./3),Np))*2
center = np.float32([0.5,0.5,0.5])
size = np.float32(1.)

from meshoid import Meshoid

M = Meshoid(np.float64(x))
plt.imshow(M.Slice(m))

Using matplotlib backend: <object object at 0x7fa1c93f8170>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:

N = 2048 
A = np.random.rand(N,N)
%timeit A @ A

import cupy

40.5 ms ± 2.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
